In [1]:
from Bio import Entrez, SeqIO
import random
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#dependency to add to README: NCBI Datasets CLI
#'conda install -c conda-forge ncbi-datasets-cli'

#if the download using CLI fails, you can use manual FTP download
#'wget -r -np -nH --cut-dirs=7 -R index.html* \
#ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/020/520/425/GCF_020520425.1/

In [3]:
#creating two variant parental genome VCF files
spinach_FASTA = "/Users/riyarampalli/spinach_genome/ncbi_dataset/data/GCF_020520425.1/GCF_020520425.1_BTI_SOV_V1_genomic.fna"

In [4]:
#function for simulating SNPs
def simulate_snps(spinach_FASTA, snp_count, output_VCF, parent_name = "Parental"):
    """
    This function simulates random SNPS along a genome's FASTA file and create an output VCF file
    arguments:
        fasta_file (str) = path to the genome file
        snp_count (int) = number of SNPs to simulate per individual
        output_vcf (str) = output path for VCF file
        parent_name (str) = name to assign to the VCF file
    """
    sequences = list(SeqIO.parse(spinach_FASTA, "fasta"))
    tot_nome_length = sum(len(seq) for seq in sequences)

    print(f"The total genome length is: {tot_nome_length:,} bp across {len(sequences)} chromosomes/contigs")

    #generate random SNP positions
    snp_positions = random.sample(range(1,tot_nome_length), snp_count) #will create snps from 1bp to total bp
    snp_positions.sort()

    snp_by_chromosome = {} #storing snps per chromosome
    nome_cursor = 0 #starting count at 0

    for seq in sequences:
        chrom = seq.id
        chrom_length = len(seq)
        chrom_snps = [] #empty list

        while snp_positions and nome_cursor < snp_positions[0] <= nome_cursor + chrom_length:
            pos_on_chrom = snp_positions[0] - nome_cursor #accesses first SNP position
            ref_base = seq.seq[pos_on_chrom - 1].upper()

            #what if the base is N or ambiguous??
            if ref_base not in "ACGT":
                continue 
            #choose an alternate base that is different from the reference
            alt_base = random.choice([b for b in "ACGT" if b != ref_base])
            chrom_snps.append((pos_on_chrom, ref_base, alt_base))

        nome_cursor += chrom_length
        if chrom_snps:
            snps_by_chromosome[chrom] = chrom_snps

    #generate VCF output 
    with open(output_VCF, "w") as vcf:
        #create headers
        vcf.write("##fileformate=VCFv.2\n")
        vcf.write(f"##source=Simulated_{parent_name}\n")
        vcf.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\t" + parent_name + "\n")

        #write the SNPs
        for chrom, snps in snps_by_chromosome.items():
            for pos, ref, alt in snps:
                vcf.write(f"{chrom}\t{pos}\t.\t{ref}\t{alt}\t.\tPASS\t.\tGT\t1/1\n")

    print(f"{parent_name} SNP VCF written to {output_VCF} with {snp_count} SNPs.\n")


In [ ]:
#defining number of SNPs to simulate per parent 
num_SNP = 1000

#simulate parental 1 and 2 SNPs
simulate_snps(spinach_FASTA, num_SNP, "parent1.vcf", parent_name = "Parental 1")
simulate_snps(spinach_FASTA, num_SNP, "parent2.vcf", parent_name = "Parental 2")

The total genome length is: 894,736,656 bp across 246 chromosomes/contigs


I want to visualize the VCF files. Some common options for visualizing VCFs are: (1) SNP density plots that allow the user to see how the SNPs are distributed across the chromosomes, (2) variant calling, and (3)genome comparisons to see differences between genome 1 and genome 2, and (4)recombination inheritance.

In [ ]:
#let's extract the exact chromosome SNPs and positions from the two VCF files

def parse_vcfs(vcf_file):
    """
    This function will allow you to parse a VCF file and return a dataframe containing chromosome numbers and positions
    """

    variants = [] #creating an empty list

    with open(vcf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue #skip because its a header
            parts = line.strip().split('\t')
            chrom = parts[0]
            pos = int(parts[1])
            ref = parts[3]
            alt = parts[4]
            variants.append([chrom, pos, ref, alt])
    df = pd.DataFrame(variants, columns = ['CHROM', 'POS', 'REF', 'ALT'])
    return df

In [ ]:
parent1_df = parse_vcfs("parent1.vcf")
parent2_df = parse_vcfs("parent2.vcf")

In [ ]:
parent1_df.head() #show first five rows of dataframe

In [ ]:
#what if you want to plot SNP density to see the spread of the SNPs?
def plot_snps(df, parent_name):
    """
    This function allows you to plot SNPs density across each chromosome
    """
    chroms = df['CHROM'].unique()

    plt.figure(figsize = (10,6))

    for chrom in chroms:
        subset = df[df['CHROM'] == chrom]
        plt.scatter(subset['POS'], [chrom] * len(subset), label = chrom, s = 10)

    plt.title(f"SNP Distribution - {parent_name}")
    plt.xlabel("Position on Chromosome")
    plt.ylabel("Chromosome")
    plt.grid(True)
    plt.show()

In [ ]:
plot_snps(parent1_df, "Parental 1")
plot_snps(parent2_df, "Parental 2")

In [ ]:
#let's compare the SNP overlaps of the two parental genomes
merge = pd.merge(parent1_df[['CHROM', 'POS']], parent2_df[['CHROM', 'POS']],
                 on = ['CHROM', 'POS'], how = 'outer', indicator = True)

#count the number of overlapping vs. unique SNPs
print(merge['_merge'].value_counts())

#plot these numbers
merge['_merge'].value_counts().plot(kind='bar')
plt.title("SNPs Unique to Parents or shared")
plt.ylabel("Number of SNPs")
plt.xticks(rotation = 0)
plt.show()

In [ ]:
# Check current working directory
print("Working directory:", os.getcwd())

# List files in that directory
print("Files here:", os.listdir())

# Check if parent1.vcf exists
print("parent1.vcf exists:", os.path.exists("parent1.vcf"))